In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# 1. Import the WeatherPy_database.csv file from your Weather_Database folder from Deliverable 1 as a DataFrame.. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clyde River,CA,2022-07-06 01:23:43,70.4692,-68.5914,41.23,75,0,5.75,clear sky
1,1,Faanui,PF,2022-07-06 01:23:43,-16.4833,-151.7500,79.97,72,0,14.76,light rain
2,2,Albany,US,2022-07-06 01:23:44,42.6001,-73.9662,69.39,93,100,7.70,overcast clouds
3,3,San Quintin,MX,2022-07-06 01:23:44,30.4833,-115.9500,68.70,78,27,15.86,scattered clouds
4,4,Khatanga,RU,2022-07-06 01:23:29,71.9667,102.5000,55.94,53,100,2.89,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria for their vacation
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Faanui,PF,2022-07-06 01:23:43,-16.4833,-151.7500,79.97,72,0,14.76,light rain
5,5,Namatanai,PG,2022-07-06 01:23:44,-3.6667,152.4333,84.00,68,100,9.22,light rain
6,6,Atuona,PF,2022-07-06 01:23:45,-9.8000,-139.0333,77.36,77,46,23.55,light rain
9,9,Butaritari,KI,2022-07-06 01:22:43,3.0707,172.7902,82.76,75,94,21.70,light rain
10,10,Bengkulu,ID,2022-07-06 01:22:55,-3.8004,102.2655,77.32,83,97,5.35,overcast clouds
13,13,Jamestown,US,2022-07-06 01:21:11,42.0970,-79.2353,77.02,83,75,10.36,broken clouds
14,14,Mao,ES,2022-07-06 01:23:46,39.8885,4.2658,77.56,73,0,11.50,clear sky
29,29,Rikitea,PF,2022-07-06 01:22:35,-23.1203,-134.9692,75.72,72,28,17.00,scattered clouds
30,30,Baker City,US,2022-07-06 01:23:50,44.7749,-117.8344,80.31,29,0,8.05,clear sky
32,32,Lahaina,US,2022-07-06 01:23:50,20.8783,-156.6825,87.80,80,44,11.07,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
# preferred_cities_df.count() or 
# preferred_cities_df.isnull().sum() or
preferred_cities_df.notnull().sum()

City_ID                245
City                   245
Country                245
Date                   245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [6]:
preferred_cities_df.shape

(245, 11)

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Faanui,PF,2022-07-06 01:23:43,-16.4833,-151.7500,79.97,72,0,14.76,light rain
5,5,Namatanai,PG,2022-07-06 01:23:44,-3.6667,152.4333,84.00,68,100,9.22,light rain
6,6,Atuona,PF,2022-07-06 01:23:45,-9.8000,-139.0333,77.36,77,46,23.55,light rain
9,9,Butaritari,KI,2022-07-06 01:22:43,3.0707,172.7902,82.76,75,94,21.70,light rain
10,10,Bengkulu,ID,2022-07-06 01:22:55,-3.8004,102.2655,77.32,83,97,5.35,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,79.97,light rain,-16.4833,-151.7500,
5,Namatanai,PG,84.00,light rain,-3.6667,152.4333,
6,Atuona,PF,77.36,light rain,-9.8000,-139.0333,
9,Butaritari,KI,82.76,light rain,3.0707,172.7902,
10,Bengkulu,ID,77.32,overcast clouds,-3.8004,102.2655,
13,Jamestown,US,77.02,broken clouds,42.0970,-79.2353,
14,Mao,ES,77.56,clear sky,39.8885,4.2658,
29,Rikitea,PF,75.72,scattered clouds,-23.1203,-134.9692,
30,Baker City,US,80.31,clear sky,44.7749,-117.8344,
32,Lahaina,US,87.80,scattered clouds,20.8783,-156.6825,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.count()

City                   245
Country                245
Max Temp               245
Current Description    245
Lat                    245
Lng                    245
Hotel Name             245
dtype: int64

In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,79.97,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
5,Namatanai,PG,84.00,light rain,-3.6667,152.4333,Seagulls Inn Namatani
6,Atuona,PF,77.36,light rain,-9.8000,-139.0333,Villa Enata
9,Butaritari,KI,82.76,light rain,3.0707,172.7902,Isles Sunset Lodge
10,Bengkulu,ID,77.32,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu


In [12]:
# Removed empty strings
hotel_df[hotel_df["Hotel Name"].str.contains("")==False]
hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,79.97,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
5,Namatanai,PG,84.00,light rain,-3.6667,152.4333,Seagulls Inn Namatani
6,Atuona,PF,77.36,light rain,-9.8000,-139.0333,Villa Enata
9,Butaritari,KI,82.76,light rain,3.0707,172.7902,Isles Sunset Lodge
10,Bengkulu,ID,77.32,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Jamestown,US,77.02,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
14,Mao,ES,77.56,clear sky,39.8885,4.2658,ARTIEM Capri
29,Rikitea,PF,75.72,scattered clouds,-23.1203,-134.9692,People ThankYou
30,Baker City,US,80.31,clear sky,44.7749,-117.8344,Bridge Street Inn
32,Lahaina,US,87.80,scattered clouds,20.8783,-156.6825,Lahaina Shores Beach Resort


In [13]:
# Step 7. Drop any rows in the hotel_df DataFrame where a hotel name is not found.
# clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,79.97,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
5,Namatanai,PG,84.00,light rain,-3.6667,152.4333,Seagulls Inn Namatani
6,Atuona,PF,77.36,light rain,-9.8000,-139.0333,Villa Enata
9,Butaritari,KI,82.76,light rain,3.0707,172.7902,Isles Sunset Lodge
10,Bengkulu,ID,77.32,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Jamestown,US,77.02,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
14,Mao,ES,77.56,clear sky,39.8885,4.2658,ARTIEM Capri
29,Rikitea,PF,75.72,scattered clouds,-23.1203,-134.9692,People ThankYou
30,Baker City,US,80.31,clear sky,44.7749,-117.8344,Bridge Street Inn
32,Lahaina,US,87.80,scattered clouds,20.8783,-156.6825,Lahaina Shores Beach Resort


In [14]:
clean_hotel_df.shape

(226, 7)

In [15]:
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 1 to 673
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 226 non-null    object 
 1   Country              226 non-null    object 
 2   Max Temp             226 non-null    float64
 3   Current Description  226 non-null    object 
 4   Lat                  226 non-null    float64
 5   Lng                  226 non-null    float64
 6   Hotel Name           226 non-null    object 
dtypes: float64(3), object(4)
memory usage: 14.1+ KB


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
hotel_info


['\n<dl>\n<dt>City</dt><dd>Faanui</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Description</dt><dd>light rain and 79.97 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Namatanai</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Description</dt><dd>light rain and 84.0 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Description</dt><dd>light rain and 77.36 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Butaritari</dd>\n<dt>Country</dt><dd>KI</dd>\n<dt>Current Description</dt><dd>light rain and 82.76 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Bengkulu</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Description</dt><dd>overcast clouds and 77.32 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Description</dt><dd>broken clouds and 77.02 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Mao</dd>\n<dt>Country</dt><dd>ES</dd>\n<dt>Current Description</dt><dd>clear sky and 77.56 °F</dd>\n</dl>\n'

In [18]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = clean_hotel_df[["Lat", "Lng"]]
locations_df

,Lat,Lng
1,-16.4833,-151.7500
5,-3.6667,152.4333
6,-9.8000,-139.0333
9,3.0707,172.7902
10,-3.8004,102.2655
...,...,...
661,10.4500,77.5500
666,17.9583,-102.2000
667,40.5719,9.6922
671,-6.5333,110.7000


In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))